In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from multiprocessing import *
import os
from matplotlib.ticker import StrMethodFormatter
import scipy
from pathos.multiprocessing import ProcessingPool as Pool

import sys
from IPython.display import clear_output

import DarkNews
from DarkNews import const 
# from analysis_dn import fit_functions as ff
# from analysis_dn import hist_plot
# from DarkNews.GenLauncher import GenLauncher
# from analysis_dn import analysis as a
# from analysis_dn import analysis_decay as a_d
# from analysis_dn import decayer

# 1. Set general parameters for fitting

### Path and Grid

In [5]:
# PATH
path = 'results/3p2_general/'

# Number of processors
workers = 8

# Grid parameters
points = 30
mzprime = 1.25
delta = np.geomspace(0.05, 10, points)
m5 = np.geomspace(0.01, 2.0, points)
m5=ff.round_sig(m5,4)

NameError: name 'ff' is not defined

### Parameters for simulation

In [4]:
ud4_def = 1.0/np.sqrt(2.)
ud5_def = 1.0/np.sqrt(2.)
gD_def = 2.
umu4_def = np.sqrt(1.0e-12)
umu5_def = np.sqrt(1.0e-12)
epsilon_def = 1e-2

v54 = gD_def * ud5_def * ud4_def
vmu5_def = gD_def * ud5_def * (umu4_def*ud4_def + umu5_def*ud5_def) / np.sqrt(1 - umu4_def**2 - umu5_def**2)

epsilon = 1e-2
def vmu5_f(umu4,umu5):
    return gD_def * ud5_def * (umu4*ud4_def + umu5*ud5_def) / np.sqrt(1 - umu4**2 - umu5**2)

r_eps = epsilon / epsilon_def


vmui_max = 1e-3
v_cut = vmu5_f(vmui_max,vmui_max)

### Data from MiniBooNE

In [5]:
data_MB_enu = hist_plot.get_data_MB(varplot='reco_Enu',loc='../')
back_MC_enu = data_MB_enu[1]
D_enu = data_MB_enu[0] + data_MB_enu[1]
sys_enu = [data_MB_enu[2], data_MB_enu[3]]

### Data for distributions

In [6]:
cols=['mzprime','m5','m4','delta','sum_w_post_smearing','v_mu5','v_54','epsilon','u_mu5/u_mu4','chi2','decay_length','N_events','scatt_det']
bin_e = np.array([0.2, 0.3, 0.375, 0.475, 0.55, 0.675, 0.8, 0.95, 1.1, 1.3, 1.5, 3.])

# 2. Running the simulation (DarkNews)

In [7]:
def run_dn(k_delta):
    
    for k_m5 in range(points):
        
        m4 = ff.round_sig(m5[k_m5] / (delta[k_delta] + 1),4)
        
        if (0 >= m5[k_m5] - m4):
            continue
        
        if int(m4) != m4:
            m4s = m4
        else:
            m4s = int(m4)
        
        if int(m5[k_m5]) != m5[k_m5]:
            m5s = m5[k_m5]
        else:
            m5s = int(m5[k_m5])
        
        try:
            df = pd.read_parquet(f'data/miniboone_fhc/3plus2/m5_{m5s}_m4_{m4s}_mzprime_{mzprime}_dirac/pandas_df.parquet', engine='pyarrow')
        except:
            try:
                gen = GenLauncher(mzprime=mzprime, m5=m5s, m4=m4s, Umu4=umu4_def, Umu5=umu5_def, UD4=ud4_def, UD5=ud5_def, gD=gD_def,epsilon=epsilon_def, neval=100000, HNLtype="dirac", experiment="miniboone_fhc",sparse=True,print_to_float32=True, pandas=False, parquet=True,loglevel="ERROR")
                gen.run()
                if (m5s - m4s >= mzprime):
                    d_length = gen.df.attrs['N5_ctau0']
                else:
                    d_length = gen.df.attrs['N5_ctau0'] / r_eps**2
                if (d_length >= 10):
                    gen = GenLauncher(mzprime=mzprime, m5=m5s, m4=m4s, Umu4=umu4_def, Umu5=umu5_def, UD4=ud4_def, UD5=ud5_def, gD=gD_def,epsilon=epsilon_def, neval=100000, HNLtype="dirac", experiment="miniboone_fhc_dirt",sparse=True,print_to_float32=True, pandas=False, parquet=True,loglevel="ERROR")
                    gen.run()
                clear_output(wait=False)
            except:
                continue

In [ ]:
pool = Pool(workers)
pool.map(run_dn,range(points))

Error! Final states have an excess in mass of -0.0005459978921999993 on top of parent particle.
Error! Final states have an excess in mass of -0.0004559978921999995 on top of parent particle.
Error! Final states have an excess in mass of -0.0003499978921999993 on top of parent particle.
Error! Final states have an excess in mass of -0.000451997892199999 on top of parent particle.
Error! Final states have an excess in mass of -0.0002259978922000002 on top of parent particle.
Error! Final states have an excess in mass of -8.099789219999918e-05 on top of parent particle.
Error! Final states have an excess in mass of -0.0002119978922000001 on top of parent particle.
Error! Final states have an excess in mass of -0.0003419978922 on top of parent particle.
Error! Final states have an excess in mass of -0.0003319978922000004 on top of parent particle.
Error! Final states have an excess in mass of -5.19978922000014e-05 on top of parent particle.
Error! Final states have an excess in mass of -0

# 3. Running the fitting

### Data to save partial results

In [ ]:
data_list = [path+f'data/chi2_{mzprime}_{k}.dat' for k in range(points)]

### Routine to generate grid

In [ ]:
def grid_generator(k_delta):
    
    data_tot = []
    
    for k_m5 in range(points):
        
        # 1. READ THE DATA FROM THE SIMULATION
        m4 = ff.round_sig(m5[k_m5] / (delta[k_delta] + 1),4)
        
        if (0 >= m5[k_m5] - m4):
            continue
        
        if int(m4) != m4:
            m4s = m4
        else:
            m4s = int(m4)
        
        if int(m5[k_m5]) != m5[k_m5]:
            m5s = m5[k_m5]
        else:
            m5s = int(m5[k_m5])
        
        dataset = f'data/miniboone_fhc/3plus2/m5_{m5s}_m4_{m4s}_mzprime_{mzprime}_dirac/pandas_df.parquet'
        dataset_dirt = f'data/miniboone_fhc_dirt/3plus2/m5_{m5s}_m4_{m4s}_mzprime_{mzprime}_dirac/pandas_df.parquet'
        
        try:
            df = pd.read_parquet(dataset, engine='pyarrow')
        except:
            continue
        
        
        # 2. COMPUTE DECAY LENGTH AND PREPARE DATA ROW
        if (m5s - m4s >= mzprime):
            decay_l = df.attrs['N5_ctau0']
        else:
            decay_l = df.attrs['N5_ctau0'] / r_eps**2
        
        data = [[mzprime,m5[k_m5],m4,delta[k_delta],0,0,v54,epsilon,0,0,decay_l,0,0]]
        
        # 3. ANALYSIS FOR EVENTS SCATTERED IN DETECTOR
        df = a_d.select_MB_decay_expo_prob(df,coupling_factor=1,l_decay_proper_cm=decay_l)
        df = a.compute_spectrum(df, EVENT_TYPE='both')
        df = df[df.reco_w>0]
        sum_w_post_smearing_det = np.abs(np.sum(df['reco_w'])) * r_eps**2
        
        # 4. READING AND ANALYSIS FOR EVENTS SCATTERED WITH DIRT
        try:
            df2 = pd.read_parquet(dataset_dirt, engine='pyarrow')
            df2 = a_d.select_MB_decay_dirt(df2,coupling_factor=1,l_decay_proper_cm=decay_l)
            df2 = a.compute_spectrum(df2, EVENT_TYPE='both')
            df2 = df2[df2.reco_w>0]
            df = pd.concat([df, df2])
        except:
            continue
        
        
        # 5. COMPUTATION OF DEFAULT NUMBER OF EVENTS AND FITTING
        sum_w_post_smearing = np.abs(np.sum(df['reco_w'])) * r_eps**2
        data[0][4] = sum_w_post_smearing
        
        if sum_w_post_smearing!=0:
            guess0 = np.sqrt(np.sqrt(560./sum_w_post_smearing)*vmu5_def/v_cut)
        else:
            guess0 = 1
        guess = np.min([guess0,1])
        theta_guess = np.arccos(guess)
        init_guess = np.array([theta_guess])
        chi2 = lambda theta: ff.chi2_MiniBooNE_2020_3p2(df, v_cut*np.cos(theta)**2,vmu5_def=vmu5_def,r_eps=r_eps)
        
        res = scipy.optimize.minimize(chi2, init_guess)
        vmu5_bf = v_cut*np.cos(res.x[0])**2
                
        # 6. WRITE THE DATA IN ROW
        data[0][5] = vmu5_bf
        data[0][8] = vmu5_bf / 2.
        data[0][9] = res.fun
        data[0][11] = (vmu5_bf / vmu5_def)**2 * sum_w_post_smearing
        data[0][12] = sum_w_post_smearing_det / sum_w_post_smearing
        
        data_tot += data
        clear_output(wait=False)
        
    # 7. PRINT PARTIAL RESULTS
    data_out = pd.DataFrame(data=data,columns=cols)
    data_out.to_csv(data_list[k_delta],sep='\t',index=False)
                
    return data_tot

In [ ]:
pool = Pool(workers)
chi2 = pool.map(grid_generator,range(points))

### Save all the results

In [ ]:
data = pd.DataFrame(data=[],columns=cols)
for k in range(points):
    try:
        data_temp = pd.read_csv(data_list[k],sep='\t')
        data = pd.concat([data, data_temp], ignore_index=True)
    except:
        continue

### Filter chi2 < 0

In [ ]:
data.to_csv(path+'/chi2_fit_3p2_original.dat',sep='\t',float_format='%.7e',index=False)
data = data[data.chi2>0]
data.to_csv(path+'/chi2_fit_3p2.dat',sep='\t',float_format='%.7e',index=False)